# Use NumPy to recognize handwritten digits

In [2]:
from PIL import Image
import numpy as np

Download the data. In this case, MNIST dataset.

In [4]:
import os
import urllib
from urllib.request import urlretrieve
dataset = 'mnist.pkl.gz'
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(a*b*100.0/c), end="")
    
if not os.path.isfile(dataset):
        origin = "https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz"
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset, reporthook=reporthook)

downloading: 100.0%

In [5]:
import gzip
import pickle
with gzip.open(dataset, 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

## Q
First, check shape and dtype of the dataset.

In [4]:
%run -i q_see_mnist_data.py

train_set[0]:      shape=(50000, 784)	 dtype=float32
train_set[1]:      shape=(50000,)	 dtype=int64
validation_set[0]: shape=(10000, 784)	 dtype=float32
validation_set[1]: shape=(10000,)	 dtype=int64
test_set[0]:       shape=(10000, 784)	 dtype=float32
test_set[1]:       shape=(10000,)	 dtype=int64


# Supervised Learning

<img src="supervised.svg" />
vs
<img src="supervised2.svg" />
See also Chinese room

In [8]:
train_X, train_y = train_set
test_X, test_y = test_set

### Look into the content. What is in `y`?

In [9]:
# Let's check the first 20 items in the training data
train_y[:20]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9],
      dtype=int64)

### What is in `X`?

In [11]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# show the first 20 samples in training X
showX(train_X[:20])

### Let's start from the simplest mehods

Always start from the obvious, simple, boring methods, 

For instance, in order to decide the digit in a handwritten sample, can we simply identify the most similar image in the training data, and assume they represents the same digit?

It may appear naive, but why not give it a shot?

But what exactly does similarity mean? You know, it is kind of tricky to define the similarity of two images. Some approaches, such as SSIM are related to human visual neurobiology and perception, and some methods like LPIPS employ neural networks. 

Again, always begin with the obvious, simple, boring methods. Let us using sqauare error to measure the differences between two images.

## Q 
How to calculate the square difference between two vectors u and v?
```python
u = train_X[0]
v = train_X[1]
```

In [8]:
%run -i q_square_error.py

86.94919
86.949183020698


## Q 
* Show test_X[0]
* Find a sample in  `train_X` which is most similar to `test_X[0]`.
* display the image of that sample
* check the corresponding  `train_y`
* see if it is the same as test_y[0]

In [9]:
%run -i q_find_nn_0.py

train_X[38620]


train_X[38620] = 7
test_y[0] = 7


## Q

Take the first 10, 100 samples from `test_X` and do the same. What is the accuracy of this method?

In [12]:
%run -i q_find_nn_10.py

test_X[0]


train_X[38620]


train_X[38620] = 7
train_X[0] = 7
test_X[1]


train_X[28882]


train_X[28882] = 2
train_X[1] = 2
test_X[2]


...

train_X[41982] = 9
train_X[7] = 9
test_X[8]


train_X[35628]


train_X[35628] = 5
train_X[8] = 5
test_X[9]


train_X[5044]


train_X[5044] = 9
train_X[9] = 9
Accuracy 1.0


## Q
What if train_X cotains only 500 samples?

use reshaping,  broadcasting to caculate the accuracy for `test_X[:100]`.

Hint: Can use `np.expand_dims` instead of `np.reshape`.

In [ ]:
# !可能會用掉太多記憶體!
#%run -i q_small_data.py
# accuracy: 85%

## Q
What about using other distance functions? e.g.  `np.abs(...).sum()`

### Use inner product instead of square difference
$$
\begin{align*}
\left\Vert \mathbf{u}-\mathbf{v}\right\Vert ^{2} & =\left(\mathbf{u}-\mathbf{v}\right)\cdot\left(\mathbf{u}-\mathbf{v}\right)\\
 & =\left\Vert \mathbf{u}\right\Vert ^{2}-2\mathbf{u}\cdot\mathbf{v}+\left\Vert \mathbf{v}\right\Vert ^{2}\\
\end{align*}
$$


In [ ]:
# normalize the data
train_X  = train_X / np.linalg.norm(train_X, axis=1, keepdims=True)
test_X  = test_X / np.linalg.norm(test_X, axis=1, keepdims=True)

In [ ]:
# matrix multiplication == pairwise computation of inner product
A = test_X @ train_X.T
print(A.shape)

In [ ]:
A.argmax(axis=1)

In [ ]:
predict_y = train_y[A.argmax(axis=1)]

In [ ]:
# Show first 20 samples of  Test X
showX(test_set[0][:20])

In [ ]:
# predicted Y
predict_y[:20]

In [ ]:
# First 20 samples of Test y
test_y[:20]

In [ ]:
# accuracy
(predict_y == test_y).mean()

### Employ PCA to reduce dimensions
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=784)
train_X = pca.fit_transform(train_set[0])
test_X = pca.transform(test_set[0])

In [18]:
np.mean((train_set[0][0]-train_set[0][1])**2)

0.110904574

In [19]:
np.mean((train_X[0]-train_X[1])**2)

0.11090599

In [20]:
m = train_set[0].mean(axis=0)
m.shape

(784,)

In [21]:
train_X[0]@train_X[1]

10.671792

In [23]:
(train_set[0][0]-m)@(train_set[0][1]-m)

10.671726

In [13]:
train_X /= np.linalg.norm(train_X, axis=1, keepdims=True)
test_X /= np.linalg.norm(test_X, axis=1, keepdims=True)

In [ ]:
# 矩陣乘法
A = test_X @ train_X.T

In [ ]:
predict_y = train_y[A.argmax(axis=1)]

In [ ]:
# 正確率
(predict_y == test_y).mean()

## Q
* Try other settings of PCA
* Check the differences
* use sklearn knn